# Nobebook for Stock Information

Description:
Notebook shows how to build your own Stock Database

In [1]:
# Importing your Library

## Standard Libraries
import pandas as pd
import numpy as np
import requests
import os
from IPython import display
import json
import enum
import sys
from datetime import datetime, timedelta

## Learning Libraries and External Libraries
from sqlalchemy import create_engine, Table, Column, ForeignKey, Integer, String, DateTime, Boolean, ARRAY, DATE, FLOAT, Enum, TIME, BigInteger
from sqlalchemy import select, insert, within_group, exists
from sqlalchemy.orm import declarative_base, relationship, Session
import mysql.connector as mysql
import sqlalchemy

## Datasource Modules
from fredapi import Fred
import wbgapi as wb


## Custom Modules
from MasterEconomist import *

# Initiation the OBJECT and CONNECTION

In [2]:
fred = Fred(api_key=os.environ['FRED_KEY'])
engine = create_engine(f"mysql+mysqlconnector://{os.environ['USER']}:\
    {os.environ['PASSWORD']}@{os.environ['PI']}/{os.environ['MAIN_DB']}" )
url = "https://www.alphavantage.co/query"

# Map the ORMs for the relevant Financial information for target stocks

In [120]:
Base = declarative_base()

class Stock_Mapper(Base):
    __tablename__ = "stock_symbol_mapping"
    symbol = Column(String(35), primary_key=True)
    company_name = Column(String(100))
    region = Column(String(30))
    currency = Column(String(3))
    asset_type = Column(String(25))
    marketOpen = Column(TIME)
    marketClose = Column(TIME)
    timezone = Column(String(10))

class ReportingTypes(enum.Enum):
    quarterlyReport = 1
    annualReport = 2

class Denominations(enum.Enum):
    k = 1
    m = 2
    b = 3


class Income_Statement(Base):
    __tablename__ = "income_statements"
    id = Column(Integer, primary_key=True)
    symbol = Column(String(35), ForeignKey('stock_symbol_mapping.symbol'))
    reporting_type = Column(Enum(ReportingTypes))
    reporting_denominations = Column(Enum(Denominations))
    fiscalDateEnding = Column(DATE) 
    reportedCurrency = Column(String(3))
    grossProfit = Column(FLOAT) 
    totalRevenue = Column(FLOAT)
    costOfRevenue = Column(FLOAT)
    costofGoodsAndServicesSold = Column(FLOAT)
    operatingIncome = Column(FLOAT) 
    sellingGeneralAndAdministrative = Column(FLOAT)
    researchAndDevelopment = Column(FLOAT)
    operatingExpenses = Column(FLOAT)
    investmentIncomeNet = Column(FLOAT)
    netInterestIncome = Column(FLOAT)
    interestIncome = Column(FLOAT)
    interestExpense = Column(FLOAT)
    nonInterestIncome = Column(FLOAT)
    otherNonOperatingIncome = Column(FLOAT)
    depreciation = Column(FLOAT)
    depreciationAndAmortization = Column(FLOAT)
    incomeBeforeTax = Column(FLOAT)
    incomeTaxExpense = Column(FLOAT)
    interestAndDebtExpense = Column(FLOAT)
    netIncomeFromContinuingOperations = Column(FLOAT)
    comprehensiveIncomeNetOfTax = Column(FLOAT)
    ebit = Column(FLOAT)
    ebitda = Column(FLOAT)
    netIncome = Column(FLOAT)

class Balance_Sheet(Base):
    __tablename__ = "balance_sheet"
    id = Column(Integer, primary_key=True)
    symbol = Column(String(35), ForeignKey('stock_symbol_mapping.symbol'))
    reporting_type = Column(Enum(ReportingTypes))
    reporting_denominations = Column(Enum(Denominations))
    fiscalDateEnding = Column(DATE)
    totalAssets = Column(FLOAT) 
    totalCurrentAssets =  Column(FLOAT)
    cashAndCashEquivalentsAtCarryingValue = Column(FLOAT)
    cashAndShortTermInvestments = Column(FLOAT)
    inventory = Column(FLOAT)
    currentNetReceivables = Column(FLOAT) 
    totalNonCurrentAssets = Column(FLOAT)
    propertyPlantEquipment = Column(FLOAT) 
    accumulatedDepreciationAmortizationPPE = Column(FLOAT) 
    intangibleAssets = Column(FLOAT) 
    intangibleAssetsExcludingGoodwill = Column(FLOAT) 
    goodwill = Column(FLOAT) 
    investments = Column(FLOAT) 
    longTermInvestments = Column(FLOAT) 
    shortTermInvestments = Column(FLOAT) 
    otherCurrentAssets = Column(FLOAT)
    otherNonCurrrentAssets = Column(FLOAT) 
    totalLiabilities = Column(FLOAT) 
    totalCurrentLiabilities = Column(FLOAT) 
    currentAccountsPayable = Column(FLOAT) 
    deferredRevenue = Column(FLOAT) 
    currentDebt = Column(FLOAT) 
    shortTermDebt = Column(FLOAT) 
    totalNonCurrentLiabilities = Column(FLOAT) 
    capitalLeaseObligations = Column(FLOAT) 
    longTermDebt = Column(FLOAT) 
    currentLongTermDebt = Column(FLOAT) 
    longTermDebtNoncurrent = Column(FLOAT) 
    shortLongTermDebtTotal = Column(FLOAT) 
    otherCurrentLiabilities = Column(FLOAT) 
    otherNonCurrentLiabilities = Column(FLOAT) 
    totalShareholderEquity = Column(FLOAT) 
    treasuryStock = Column(FLOAT) 
    retainedEarnings = Column(FLOAT) 
    commonStock = Column(FLOAT) 
    commonStockSharesOutstanding = Column(FLOAT) 

class CashFlow(Base):
    __tablename__ = "cashflow"
    id = Column(Integer, primary_key=True)
    symbol = Column(String(35), ForeignKey('stock_symbol_mapping.symbol'))
    reporting_type = Column(Enum(ReportingTypes))
    reporting_denominations = Column(Enum(Denominations))
    fiscalDateEnding = Column(DATE)
    operatingCashflow = Column(FLOAT) 
    paymentsForOperatingActivities = Column(FLOAT) 
    proceedsFromOperatingActivities = Column(FLOAT) 
    changeInOperatingLiabilities = Column(FLOAT) 
    changeInOperatingAssets = Column(FLOAT) 
    depreciationDepletionAndAmortization = Column(FLOAT) 
    capitalExpenditures = Column(FLOAT) 
    changeInReceivables = Column(FLOAT) 
    changeInInventory = Column(FLOAT) 
    profitLoss = Column(FLOAT) 
    cashflowFromInvestment = Column(FLOAT) 
    cashflowFromFinancing = Column(FLOAT) 
    proceedsFromRepaymentsOfShortTermDebt = Column(FLOAT) 
    paymentsForRepurchaseOfCommonStock = Column(FLOAT) 
    paymentsForRepurchaseOfEquity = Column(FLOAT) 
    paymentsForRepurchaseOfPreferredStock = Column(FLOAT) 
    dividendPayout = Column(FLOAT) 
    dividendPayoutCommonStock = Column(FLOAT) 
    dividendPayoutPreferredStock = Column(FLOAT) 
    proceedsFromIssuanceOfCommonStock = Column(FLOAT) 
    proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet = Column(FLOAT) 
    proceedsFromIssuanceOfPreferredStock = Column(FLOAT) 
    proceedsFromRepurchaseOfEquity = Column(FLOAT) 
    proceedsFromSaleOfTreasuryStock = Column(FLOAT) 
    changeInCashAndCashEquivalents = Column(FLOAT) 
    changeInExchangeRate = Column(FLOAT) 
    netIncome = Column(FLOAT) 


class Stock_Meta(Base):
    __tablename__ = "stock_meta"
    symbol = Column(String(35), primary_key=True)
    assetSubType = Column(String(50))
    companyFullName = Column(String(255))
    cik = Column(Integer)
    exchange = Column(String(30))
    currency = Column(String(3))
    country = Column(String(3))
    sector = Column(String(50))
    industry = Column(String(150))
    address = Column(String(150))
    fiscalYearEnd = Column(String(30))

class Earnings(Base):
    __tablename__ = "earnings"
    id = Column(Integer, primary_key=True)
    symbol = Column(String(35), ForeignKey('stock_symbol_mapping.symbol'))
    reporting_type = Column(Enum(ReportingTypes))
    reporting_denominations = Column(Enum(Denominations))
    fiscalDateEnding = Column(DATE)
    reportedDate = Column(FLOAT)
    reportedEPS = Column(FLOAT)
    estimatedEPS = Column(FLOAT) 
    annualEPS = Column(FLOAT)
    surprise = Column(FLOAT)
    surprisePercentage = Column(FLOAT)  
 
 

In [3]:
denom_mapper = {'b': 1_000_000_000, 'm': 1_000_000, 'k': 1_000}

In [146]:
output2['quarterlyEarnings']

[{'fiscalDateEnding': '2022-03-31',
  'reportedDate': '2022-04-14',
  'reportedEPS': 'None',
  'estimatedEPS': '1.57',
  'surprise': 'None',
  'surprisePercentage': 'None'},
 {'fiscalDateEnding': '2021-12-31',
  'reportedDate': '2022-01-14',
  'reportedEPS': '1.46',
  'estimatedEPS': '1.38',
  'surprise': '0.08',
  'surprisePercentage': '5.7971'},
 {'fiscalDateEnding': '2021-09-30',
  'reportedDate': '2021-10-14',
  'reportedEPS': '2.15',
  'estimatedEPS': '1.7852',
  'surprise': '0.3648',
  'surprisePercentage': '20.4347'},
 {'fiscalDateEnding': '2021-06-30',
  'reportedDate': '2021-07-14',
  'reportedEPS': '2.84',
  'estimatedEPS': '1.8908',
  'surprise': '0.9492',
  'surprisePercentage': '50.201'},
 {'fiscalDateEnding': '2021-03-31',
  'reportedDate': '2021-04-15',
  'reportedEPS': '3.62',
  'estimatedEPS': '2.5309',
  'surprise': '1.0891',
  'surprisePercentage': '43.0321'},
 {'fiscalDateEnding': '2020-12-31',
  'reportedDate': '2021-01-15',
  'reportedEPS': '2.07',
  'estimatedEPS

In [145]:
output2['annualEarnings']

[{'fiscalDateEnding': '2022-03-31', 'reportedEPS': '0'},
 {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '10.07'},
 {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '5.04'},
 {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '7.57'},
 {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '6.65'},
 {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '5.0329'},
 {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '4.72'},
 {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '5.43'},
 {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '3.75'},
 {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '4.38'},
 {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '3.86'},
 {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '3.7'},
 {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '3.5'},
 {'fiscalDateEnding': '2009-12-31', 'reportedEPS': '-2.9'},
 {'fiscalDateEnding': '2008-12-31', 'reportedEPS': '-46'},
 {'fiscalDateEnding': '2007-12-31', 'reportedEPS': '7'},
 {'fiscalDateEnding': '2006-12-31', 'reportedE

In [118]:
session.rollback()

In [119]:
session.execute("drop table stock_meta")

In [83]:
abs(int(output['annualReports'][0]['incomeTaxExpense']))/1_000_000_000

0.743

In [121]:
Base.metadata.create_all(engine)

In [22]:
with Session(engine) as session:
    session.execute('drop table stock_symbol_mapping')

In [4]:
# Institution Level / Bluechip companies
tier1_searchTerms = [ 'McKesson', 'Exxon Mobil',' AmerisourceBergen', 
 'Berkshire Hathaway']

tier2_searchTerms = ['Costco', 'ebay', 'gamestop', 'Pepco', 'Home Depot', 'Target', 'Pfizer', 'Boeing', 'Ford', 'General Motors', 'ConocoPhilips', 
'Bank of America', 'Hewlett-Packard', 'Sears Holding', 'PepsiCo', 'MetLife', 'Kraft Foods', 'Dell Inc', 'Best Buy', 'Freddie Mac', 'Nordstrom, Inc', 'New York Life Insurance', 'Visa', 'Fannie Mae',
'Delta Airlines', 'United Airlines', 'American International Group', 'Proctor & Gamble', 'DuPont', 'United Parcel Service', 'Yellow Freight System', 'Schneider', 'Roadway Express'
]

payload = {
    "function" : "SYMBOL_SEARCH",
    # "maturity" :  '30year',
    "apikey": os.environ['ALPHA_VANTAGE']
}
# with Session(engine) as session:
    # for company in tier2_searchTerms:
    #     payload['keywords'] = company
    #     r = requests.get(url, payload)
    #     output = json.loads(r.text)
    #     for stock in output['bestMatches'][:5]:
    #         session.add(Stock_Mapper(symbol = stock['1. symbol'], company_name = stock['2. name'], region = stock['4. region'], currency = stock['8. currency'], asset_type = stock['3. type'], marketOpen = stock['5. marketOpen'], marketClose= stock['6. marketClose'], timezone = stock['7. timezone']))
    #     session.commit() 
    #     print("Finished", company)   
    #     time.sleep(13)



In [41]:
session = Session(engine)

In [110]:
))

In [109]:
session.query(Stock_Meta).filter( ~ exists().where(Stock_Mapper.symbol == Stock_Meta.symbol)).all()

[]

In [113]:
session.execute(select(Stock_Meta)).all()

[]

In [116]:
output['Description'][:255]

"Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2"

In [122]:
payload['function'] = 'OVERVIEW'
for val in session.execute(select(Stock_Mapper.symbol).filter(Stock_Mapper.region == 'United States').filter(Stock_Mapper.asset_type=='Equity')).all():
    payload['symbol'] = val[0]
    r = requests.get(url, payload)
    output = json.loads(r.text)
    if bool(output):
        session.add(Stock_Meta(symbol = output['Symbol'], assetSubType = output['AssetType'], 
        companyFullName=output['Name'], cik = output['CIK'],
        exchange = output['Exchange'], currency = output['Currency'], country = output ['Country'], sector = output['Sector'], industry = output['Industry'], address = output['Address'],
        fiscalYearEnd = output['FiscalYearEnd']
        ))
        session.commit()
    time.sleep(13)
    
    

In [101]:
val

('CAKFF',)

In [22]:
def adjust_output(output, denom_mapper, mode='annualReports'):
    for report in output[mode]:
        takeBreak = False
        annualReport = {}
        for key, val in denom_mapper.items():
            for col, value in report.items():
                try:
                    newNum = float(value)/val
                    annualReport[col] = newNum
                    if newNum > 1:
                        takeBreak = True
                except:
                    annualReport[col] = value
            if bool(takeBreak):
                annualReport['reporting_denominations'] = key
                annualReport['reporting_type'] = ReportingTypes._member_map_[mode]
                break
        yield annualReport

In [16]:
ReportingTypes._member_map_

{'quarterlyReports': <ReportingTypes.quarterlyReports: 1>,
 'annualReports': <ReportingTypes.annualReports: 2>}

In [11]:
[annual_statement for annual_statement in adjust_output(output, denom_mapper)][0]

{'fiscalDateEnding': '2021-12-31',
 'reportedCurrency': 'USD',
 'grossProfit': 273.474,
 'totalRevenue': 285.64,
 'costOfRevenue': 12.166,
 'costofGoodsAndServicesSold': 211.806,
 'operatingIncome': 261.404,
 'sellingGeneralAndAdministrative': 9.574,
 'researchAndDevelopment': 0.843,
 'operatingExpenses': 12.07,
 'investmentIncomeNet': 0.033,
 'netInterestIncome': -0.947,
 'interestIncome': 5.71,
 'interestExpense': 0.947,
 'nonInterestIncome': 276.674,
 'otherNonOperatingIncome': 2.291,
 'depreciation': 'None',
 'depreciationAndAmortization': 20.607,
 'incomeBeforeTax': 30.676,
 'incomeTaxExpense': 7.636,
 'interestAndDebtExpense': 0.947,
 'netIncomeFromContinuingOperations': 23.598,
 'comprehensiveIncomeNetOfTax': 25.981,
 'ebit': 31.623,
 'ebitda': 52.23,
 'netIncome': 23.04,
 'reporting_denominations': 'b',
 'reporting_type': 'annualReports'}

<Denominations.k: 1>

In [29]:
with Session(engine) as session:
    print(session.execute("SHOW TABLES").all())

[('broadcaster',), ('game_meta',), ('income_statements',), ('real_gdp',), ('sov_yc',), ('stock_meta',), ('stock_symbol_mapping',), ('stream_capture',), ('tag_mapper',), ('tag_stream_association',)]


In [24]:
payload['function'] = "INCOME_STATEMENT"
with Session(engine) as session:
    for row in session.execute(select(Stock_Meta.symbol).filter(Stock_Meta.assetSubType.ilike('%Common%'))).all():
        payload['symbol'] = row[0]
        r = requests.get(url, payload)
        output = json.loads(r.text)
    if bool(output):
        for annualReport in adjust_output(output, denom_mapper):
            session.add(Income_Statement(**annualReport))
        for quarterlyReport in adjust_output(output, denom_mapper):
            session.add(Income_Statement(**quarterlyReport))
        session.commit()
        print("Finished", payload['symbol'])
        time.sleep(13)
            
                





NoReferencedTableError: Foreign key associated with column 'income_statements.symbol' could not find table 'stock_symbol_mapping' with which to generate a foreign key to target column 'symbol'

In [8]:
r = requests.get(url, payload)
output = json.loads(r.text)

In [16]:
for i in range(10):
    for n in range(8):
        if n> 2:
            break
    if i > 8:
        print(i)


9


In [12]:
id = Column(Integer, primary_key=True)
    symbol = Column(String(35), ForeignKey('stock_symbol_mapping.symbol'))
    reporting_type = Column(Enum(ReportingTypes))
    reporting_denominations = Column(Enum(Denominations))
    fiscalDateEnding = Column(DATE) 
    reportedCurrency = Column(String(3))
 

{'symbol': 'XOM',
 'annualReports': [{'fiscalDateEnding': '2021-12-31',
   'reportedCurrency': 'USD',
   'grossProfit': '273474000000',
   'totalRevenue': '285640000000',
   'costOfRevenue': '12166000000',
   'costofGoodsAndServicesSold': '211806000000',
   'operatingIncome': '261404000000',
   'sellingGeneralAndAdministrative': '9574000000',
   'researchAndDevelopment': '843000000',
   'operatingExpenses': '12070000000',
   'investmentIncomeNet': '33000000',
   'netInterestIncome': '-947000000',
   'interestIncome': '5710000000',
   'interestExpense': '947000000',
   'nonInterestIncome': '276674000000',
   'otherNonOperatingIncome': '2291000000',
   'depreciation': 'None',
   'depreciationAndAmortization': '20607000000',
   'incomeBeforeTax': '30676000000',
   'incomeTaxExpense': '7636000000',
   'interestAndDebtExpense': '947000000',
   'netIncomeFromContinuingOperations': '23598000000',
   'comprehensiveIncomeNetOfTax': '25981000000',
   'ebit': '31623000000',
   'ebitda': '52230000

In [188]:
output.keys()

dict_keys(['symbol', 'annualReports', 'quarterlyReports'])

In [138]:
payload['function'] = 'EARNINGS'
payload['symbol'] = 'C'
r = requests.get(url, payload)
output2 = json.loads(r.text)

In [187]:
payload['function'] = 'INCOME_STATEMENT'
payload['symbol'] = 'BA'
r = requests.get(url, payload)
output = json.loads(r.text)

In [156]:
for key in output['annualReports'][0].keys():
    if key not in list(output['annualReports'][0].keys()):
        print(key)

In [153]:
for key in output['quarterlyEarnings'][0].keys():
    if key not in list(output['annualEarnings'][0].keys()):
        print(key)

reportedDate
estimatedEPS
surprise
surprisePercentage


In [66]:
output2['annualReports'][0]

{'fiscalDateEnding': '2021-12-31',
 'reportedCurrency': 'USD',
 'grossProfit': '26678000000',
 'totalRevenue': '71884000000',
 'costOfRevenue': '45206000000',
 'costofGoodsAndServicesSold': 'None',
 'operatingIncome': '14480000000',
 'sellingGeneralAndAdministrative': '11632000000',
 'researchAndDevelopment': 'None',
 'operatingExpenses': '12198000000',
 'investmentIncomeNet': 'None',
 'netInterestIncome': '42494000000',
 'interestIncome': '50475000000',
 'interestExpense': '7981000000',
 'nonInterestIncome': '29390000000',
 'otherNonOperatingIncome': '1165000000',
 'depreciation': 'None',
 'depreciationAndAmortization': '360000000',
 'incomeBeforeTax': '27403000000',
 'incomeTaxExpense': '5451000000',
 'interestAndDebtExpense': '7981000000',
 'netIncomeFromContinuingOperations': '21945000000',
 'comprehensiveIncomeNetOfTax': '15245000000',
 'ebit': '35384000000',
 'ebitda': '35744000000',
 'netIncome': '21952000000'}

In [ ]:
{: 'MSFT',
   : 'Microsoft Corporation',
   : 'Equity',
   : 'United States',
   : '09:30',
   : '16:00',
   : 'UTC-04',
   : 'USD',
   '9. matchScore': '0.6154'},